In [ ]:
!pip install findspark


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=651c875fad92a4541a5c1aa63de334b716450ae7ca013bcc31800fdfbaa9d275
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.

# Add the file to Spark context
spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows of the DataFrame to verify the data
df.show()

# Optionally, print the schema to understand the structure of the DataFrame
df.printSchema()

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

# Verify that the temporary view has been created by running a simple query
spark.sql("SELECT * FROM home_sales LIMIT 10").show()

# Stop the Spark session when done
# spark.stop()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    year
ORDER BY
    year
"""

result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

result = spark.sql(query)
result.show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?


query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to find the average price of a home per "view" rating, rounded to two decimal places,
# for homes with an average price greater than or equal to $350,000, ordered by descending view rating
query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

# End time measurement and print the runtime
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.770909070968628 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.

# Cache the temporary table home_sales
spark.sql("CACHE TABLE home_sales")

# Verify that the table is cached
spark.sql("SELECT * FROM home_sales").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached('home_sales')

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached?", is_cached)


Is home_sales cached? True


In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# SQL query to calculate the average price of a home per "view" rating, rounded to two decimal places,
# for homes with an average price greater than or equal to $350,000, ordered by descending view rating
query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

# End time measurement and print the runtime
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.5537176132202148 seconds ---


In [ ]:
# Get a list of all tables in the Spark catalog
tables = spark.catalog.listTables()

# Display the list of tables
for table in tables:
    print(table)


Table(name='home_sales', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Read the table into a DataFrame
home_sales_df = spark.table("home_sales")

home_sales_df.write.partitionBy('date_built').parquet('p_home_sales', mode='overwrite')

In [ ]:
# 11. Read the formatted parquet data.
# Read the formatted Parquet data
formatted_parquet_data = spark.read.parquet("p_home_sales")


In [ ]:
# 12. Create a temporary table for the parquet data.

formatted_parquet_data.createOrReplaceTempView("parquet_data")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()


# SQL query to calculate the average price of a home per "view" rating, rounded to two decimal places,
# for homes with an average price greater than or equal to $350,000, ordered by descending view rating
query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM
    parquet_data
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result_parquet = spark.sql(query)

# Show the result
result_parquet.show()

# End time measurement and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9302337169647217 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.

spark.catalog.clearCache()


In [ ]:
# 15. Check if the home_sales is no longer cached

# Check if the table is cached
is_cached = spark.catalog.isCached('home_sales')

# Print the result
print("Is home_sales cached?", is_cached)


Is home_sales cached? False
